Importing libraries and importing the DF

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
Toronto_df = pd.read_csv('Tor_FINAL.csv', index_col = 0)

In [3]:
Toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Finding coordinates for the city of toronto, selecting the Borough with the word 'Toronto? in it and finally printing a map with the pins from the coordinates of the database

In [4]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent = 'my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

In [6]:
toronto_data = Toronto_df[Toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [7]:
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'],
                                           toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_toronto)  
    
map_toronto

Setting the variables and creating a function for retrieving the Venues in a radius of 1000 mt from the center of the Borough, and then putting the result in a DF

In [1]:
CLIENT_ID = 'WFU4QVT5P2VFOGGHWIT1N2IFDKK15VR2LNBUXTH4GJ1JSMBQ'
CLIENT_SECRET = 'E5HD1RAH4PVBJ403KLILAI0BU4MY0ZMRYEBRKR0EIBBZBGVE'
VERSION = '20180605'

def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT = 100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [2]:
Toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude'])

NameError: name 'toronto_data' is not defined

In [33]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
1,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
4,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub


In [34]:
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix = '', prefix_sep = '')
toronto_onehot = toronto_onehot.drop(columns = ['Neighborhood'])
toronto_onehot.insert(loc = 0, column = 'Neighborhood', value = Toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tree,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Grouping by Neighborhood the DF, creating a function to sort and select the best venues in every Neighborhood

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tree,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,"Adelaide, King, Richmond",0.00,0.00,0.00,0.000000,0.000000,0.040000,0.00,0.00,0.00,0.00,0.010000,0.01,0.01,0.010000,0.000000,0.00,0.000000,0.000000,0.020000

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Hotel,Coffee Shop,American Restaurant,Theater,Restaurant,Steakhouse,Concert Hall,Bar,Sushi Restaurant
1,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Cocktail Bar,Park,Japanese Restaurant,Restaurant,Italian Restaurant,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Restaurant,Bar,Furniture / Home Store,Bakery,Vegetarian / Vegan Restaurant,Indian Restaurant,Arts & Crafts Store,Italian Restaurant
3,Business reply mail Processing Centre969 Eastern,Park,Pizza Place,Coffee Shop,Brewery,Pet Store,Burrito Place,Italian Restaurant,Pub,Sushi Restaurant,Breakfast Spot
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Airport,Airport Lounge,Sculpture Garden,Track,Dance Studio,Garden
5,"Cabbagetown, St. James Town",Café,Park,Gastropub,Restaurant,Diner,Italian Restaurant,Indian Restaurant,Japanese Restaurant,Pub,Theater
6,Central Bay Street,Coffee Shop,Café,Ramen Restaurant,Bubble Tea Shop,Bar,Japanese Restaurant,Arts & Crafts Store,Tea Room,Plaza,Pizza Place
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,Ice Cream Shop,Mexican Restaurant,Dumpling Restaurant
8,Christie,Korean Restaurant,Coffee Shop,Café,Grocery Store,Cocktail Bar,Indian Restaurant,Ice Cream Shop,Pub,Bar,Ethiopian Restaurant
9,Church and Wellesley,Coffee Shop,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Sushi Restaurant,Café,Gay Bar,Diner,Italian Restaurant,Restaurant


Dividing into cluster and printing the results on a map

In [38]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
toronto_merged = toronto_data

toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Coffee Shop,Pub,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Bakery,Burger Joint,Tea Room
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Café,Pub,Ice Cream Shop,Spa,Fast Food Restaurant,Pizza Place,Grocery Store,Ramen Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Indian Restaurant,Coffee Shop,Café,Beach,Burrito Place,Bakery,Park,Brewery,Sandwich Place,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Coffee Shop,Café,Bakery,Bar,American Restaurant,Italian Restaurant,Vietnamese Restaurant,Diner,Brewery,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Bus Line,College Gym,Café,Bookstore,College Quad,Gym / Fitness Center,Trail,Coffee Shop,Park,Eastern European Restaurant


In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []

for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],
                                  toronto_merged['Longitude'],
                                  toronto_merged['Neighborhood'],
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

An inside look at the clusters

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,0,Café,Park,Gastropub,Restaurant,Diner,Italian Restaurant,Indian Restaurant,Japanese Restaurant,Pub,Theater
12,Downtown Toronto,0,Coffee Shop,Burger Joint,Japanese Restaurant,Bubble Tea Shop,Sushi Restaurant,Café,Gay Bar,Diner,Italian Restaurant,Restaurant
20,Downtown Toronto,0,Hotel,Coffee Shop,Café,Steakhouse,Italian Restaurant,Concert Hall,American Restaurant,Restaurant,Deli / Bodega,Park
21,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Gastropub,Italian Restaurant,American Restaurant,Steakhouse,Concert Hall,Deli / Bodega
25,Downtown Toronto,0,Café,Bakery,Bar,Vegetarian / Vegan Restaurant,Restaurant,Pizza Place,Coffee Shop,Bookstore,Mexican Restaurant,Pub
26,Downtown Toronto,0,Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Coffee Shop,Chinese Restaurant,Vietnamese Restaurant,Ice Cream Shop,Mexican Restaurant,Dumpling Restaurant
29,Downtown Toronto,0,Hotel,Coffee Shop,Café,American Restaurant,Steakhouse,Deli / Bodega,Restaurant,Italian Restaurant,Theater,Thai Restaurant
35,West Toronto,0,Coffee Shop,Bar,Café,Pizza Place,Breakfast Spot,Sushi Restaurant,Thai Restaurant,Sandwich Place,Restaurant,Eastern European Restaurant
36,West Toronto,0,Coffee Shop,Café,Bakery,Italian Restaurant,Falafel Restaurant,Pizza Place,Pub,Spa,Sushi Restaurant,Shoe Store
37,East Toronto,0,Park,Pizza Place,Coffee Shop,Brewery,Pet Store,Burrito Place,Italian Restaurant,Pub,Sushi Restaurant,Breakfast Spot


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Coffee Shop,Pub,Pizza Place,Breakfast Spot,Beach,Japanese Restaurant,Bar,Bakery,Burger Joint,Tea Room
1,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Pub,Ice Cream Shop,Spa,Fast Food Restaurant,Pizza Place,Grocery Store,Ramen Restaurant
2,East Toronto,1,Indian Restaurant,Coffee Shop,Café,Beach,Burrito Place,Bakery,Park,Brewery,Sandwich Place,Fast Food Restaurant
5,Central Toronto,1,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Café,Sushi Restaurant,Pizza Place,Gym,Thai Restaurant,Dessert Shop,Sporting Goods Shop
6,Central Toronto,1,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Sushi Restaurant,Skating Rink,Thai Restaurant,Café,Mexican Restaurant,Diner,Chinese Restaurant
7,Central Toronto,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Thai Restaurant,Café,Restaurant,Mexican Restaurant,Dessert Shop,Gym
8,Central Toronto,1,Italian Restaurant,Coffee Shop,Gym,Park,Café,Sushi Restaurant,Bagel Shop,Grocery Store,Japanese Restaurant,Pharmacy
9,Central Toronto,1,Coffee Shop,Park,Sushi Restaurant,Italian Restaurant,Café,Gym,Thai Restaurant,French Restaurant,Pizza Place,Bagel Shop
10,Downtown Toronto,1,Park,Bank,Hostel,Trail,Convenience Store,Playground,Coffee Shop,Office,Sandwich Place,Candy Store
14,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Restaurant,Ramen Restaurant,Gastropub,Italian Restaurant,Tea Room,Japanese Restaurant,Cosmetics Shop


In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Sushi Restaurant,Bank,Café,Pharmacy,Italian Restaurant,Coffee Shop,Bakery,Gastropub,Japanese Restaurant,Asian Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Bus Line,College Gym,Café,Bookstore,College Quad,Gym / Fitness Center,Trail,Coffee Shop,Park,Eastern European Restaurant


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Coffee Shop,Café,Bakery,Bar,American Restaurant,Italian Restaurant,Vietnamese Restaurant,Diner,Brewery,Sandwich Place
13,Downtown Toronto,4,Coffee Shop,Café,Italian Restaurant,Theater,Bakery,Diner,Sushi Restaurant,Restaurant,Breakfast Spot,Park
15,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Cosmetics Shop,Breakfast Spot,American Restaurant,Gastropub
17,Downtown Toronto,4,Coffee Shop,Café,Ramen Restaurant,Bubble Tea Shop,Bar,Japanese Restaurant,Arts & Crafts Store,Tea Room,Plaza,Pizza Place
24,Central Toronto,4,Café,Vegetarian / Vegan Restaurant,Italian Restaurant,Restaurant,Bakery,History Museum,Grocery Store,Pizza Place,Coffee Shop,Pub
27,Downtown Toronto,4,Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Airport,Airport Lounge,Sculpture Garden,Track,Dance Studio,Garden
28,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Hotel,Cocktail Bar,Japanese Restaurant,Gastropub,Bakery,Steakhouse,Seafood Restaurant
